In [1]:
# import
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
# inline for testing
%matplotlib inline

In [3]:
# Physician
physician = pd.read_csv('..\data\Physician_&_Other_Supplier_Payments\Medicare_Provider_Util_Payment_PUF_CY2017.txt', 
                           sep = "\t",
                           usecols = ['npi', 'nppes_provider_last_org_name','nppes_provider_first_name', 'nppes_credentials', 'nppes_entity_code', 'place_of_service', 'nppes_provider_zip', 'nppes_provider_state', 'hcpcs_code','bene_day_srvc_cnt', 'average_Medicare_allowed_amt', 'provider_type'],
#                            chunksize = 100000,
                            skiprows =[1],
                           dtype = {
                               'nppes_provider_zip': 'str'
                           })
physician.columns = physician.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [4]:
# Hospital 
# create filepaths 
hospital = pd.read_csv('..\data\Hospital_Outpatient\Provider_Outpatient_Hospital_Charge_Data_by_APC__CY2017.csv',
                          usecols=[0,1,3,4,5,7,8,10,12],
                          dtype = {
                              'Provider_Zip_Code': 'str',
                              'APC': 'str'
                          })
# if it breaks, take out col 1
hospital.columns = hospital.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [5]:
# ZIP
cbsa = pd.read_csv('..\data\Zip_Code_to_CBSA\ZIP_CBSA_032020.csv',
                  usecols =['ZIP', 'CBSA'],
                  dtype ={
                      'ZIP': 'str',
                      'CBSA': 'str'
                  })

In [6]:
# Crosswalk
crosswalk = pd.read_csv('..\\data\\APC_to_CPTHCPCS_crosswalk\\2020_january_web_addendum_b.12312019.csv', 
                        header=2, 
                        dtype ={
                            'APC': 'str'
                        })
# crosswalk = crosswalk.rename(columns=lambda x: x.strip())
crosswalk.columns = crosswalk.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [7]:
#zip to 5 digits
physician['nppes_provider_zip'] = physician['nppes_provider_zip'].str[:5]

Get every zipcode to have a CBSA

In [8]:
physician.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_credentials,nppes_entity_code,nppes_provider_zip,nppes_provider_state,provider_type,place_of_service,hcpcs_code,bene_day_srvc_cnt,average_medicare_allowed_amt
0,1003000126,ENKESHAFI,ARDALAN,M.D.,I,21502,MD,Internal Medicine,F,99217,100,73.398800
1,1003000126,ENKESHAFI,ARDALAN,M.D.,I,21502,MD,Internal Medicine,F,99218,26,100.080000
2,1003000126,ENKESHAFI,ARDALAN,M.D.,I,21502,MD,Internal Medicine,F,99219,52,136.380000
3,1003000126,ENKESHAFI,ARDALAN,M.D.,I,21502,MD,Internal Medicine,F,99220,59,190.363729
4,1003000126,ENKESHAFI,ARDALAN,M.D.,I,21502,MD,Internal Medicine,F,99221,16,101.680000


In [9]:
# Assign every zipcode a CBSA
physician_with_cbsa = pd.merge(physician, cbsa, how='left', left_on='nppes_provider_zip', right_on='ZIP')

In [10]:
physician_with_cbsa = physician_with_cbsa.drop('ZIP', axis =1)

In [11]:
# physician_with_cbsa

This is every state

In [12]:
# Assign every hcpcs code an APC
physician_full_merge = pd.merge(physician_with_cbsa, crosswalk[['hcpcs_code', 'apc']], on='hcpcs_code', how='left')

In [13]:
# Get CBSAs for Nashville
cbsa_34980 =cbsa[cbsa['CBSA']== '34980']

# and New York
cbsa_35620 = cbsa[cbsa['CBSA'] == '35620']

# Chicago 16980


In [14]:
# Create variable holding all unique zip codes within CBSA
# Nash
zip_34980 = cbsa_34980['ZIP'].unique()

# New York
zip_35620 = cbsa_35620['ZIP'].unique()

In [15]:
# Seperator
# Seperator
# Seperator
# Seperator
# Seperator
# Seperator

In [16]:
# Narrow down Entity Type to O, Provider to ASC, CPt to 43249
# This is NATION wide
physician_filtered_opt1 = physician_with_cbsa[(physician_with_cbsa['nppes_entity_code'] == 'O') & (physician_with_cbsa['provider_type'] == 'Ambulatory Surgical Center')]

In [17]:
# Let's narrow it down to just Nashville 
# This is ALL Nashville providers and procedures
nash_provider = physician_filtered_opt1[physician_filtered_opt1.nppes_provider_zip.isin(zip_34980)]

In [18]:
# Narrow down to CPT 43249
nash_prov_43249 = nash_provider[nash_provider['hcpcs_code'] == '43249']

In [19]:
nash_prov_43249 = nash_prov_43249.drop(['nppes_entity_code', 'place_of_service', 'nppes_provider_state'], axis=1)

USE THIS TABLE FOR NASHVILLE PROVIDERS

In [20]:
nash_prov_43249

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_credentials,nppes_provider_zip,provider_type,hcpcs_code,bene_day_srvc_cnt,average_medicare_allowed_amt,CBSA
1419489,1124498357,ENDOSCOPY CENTER OF INLAND EMPIRE LLC,NaN,NaN,37219,Ambulatory Surgical Center,43249,16,660.990000,34980
2434815,1225039761,ENDOSCOPY CENTER OF ST THOMAS LP,NaN,NaN,37205,Ambulatory Surgical Center,43249,17,565.636471,34980
3211255,1295708964,COLUMBIA TN ENDOSCOPY ASC LLC,NaN,NaN,38401,Ambulatory Surgical Center,43249,34,587.110000,30280
3211256,1295708964,COLUMBIA TN ENDOSCOPY ASC LLC,NaN,NaN,38401,Ambulatory Surgical Center,43249,34,587.110000,34980
4676285,1427152073,PATIENT PARTNERS LLC,NaN,NaN,37066,Ambulatory Surgical Center,43249,23,587.110000,34980
6997001,1639299803,MURFREESBORO MEDICAL CLINIC P A,NaN,NaN,37129,Ambulatory Surgical Center,43249,33,587.110000,34980


In [21]:
# Let's narrow it down to just NY 
# This is ALL NY providers and procedures
ny_provider = physician_filtered_opt1[physician_filtered_opt1.nppes_provider_zip.isin(zip_35620)]

In [22]:
# Narrow down to CPT 43249
ny_prov_43249 = ny_provider[ny_provider['hcpcs_code'] == '43249']

In [23]:
ny_prov_43249 = ny_prov_43249.drop(['nppes_entity_code', 'place_of_service', 'nppes_provider_state'], axis=1)

USE THIS TABLE FOR NEW YORK PROVIDERS

In [24]:
ny_prov_43249

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_credentials,nppes_provider_zip,provider_type,hcpcs_code,bene_day_srvc_cnt,average_medicare_allowed_amt,CBSA
1406803,1124324181,MANHATTAN ENDOSCOPY CENTER LLC,NaN,NaN,10017,Ambulatory Surgical Center,43249,33,690.190909,35620
1580610,1144291261,"OCEAN SURGICAL PAVILION, PC",NaN,NaN,07755,Ambulatory Surgical Center,43249,11,696.400000,35620
1961060,1174873780,"YORKVILLE ENDOSCOPY, LLC",NaN,NaN,10128,Ambulatory Surgical Center,43249,28,634.541429,35620
2264921,1205875283,RWJ ENDOSURGICAL CENTER LLC,NaN,NaN,08816,Ambulatory Surgical Center,43249,17,696.400000,35620
2323393,1215134564,"EMMAUS SURGICAL CENTER, LLC",NaN,NaN,07840,Ambulatory Surgical Center,43249,14,583.460000,10900
2323394,1215134564,"EMMAUS SURGICAL CENTER, LLC",NaN,NaN,07840,Ambulatory Surgical Center,43249,14,583.460000,35620
2561010,1235178146,"SUBURBAN ENDOSCOPY CENTER, LLC",NaN,NaN,07044,Ambulatory Surgical Center,43249,12,641.450000,35620
2626485,1235551052,"LONG ISLAND DIGESTIVE ENDOSCOPY CENTER, LLC",NaN,NaN,11776,Ambulatory Surgical Center,43249,15,696.400000,35620
5144601,1467696815,MORRIS COUNTY SURGICAL CENTER LLC,NaN,NaN,07054,Ambulatory Surgical Center,43249,13,641.450000,35620
5211148,1477584621,HANOVER NJ ENDOSCOPY ASC LLC,NaN,NaN,07981,Ambulatory Surgical Center,43249,41,641.450000,35620


Get the same result for hospitals

In [25]:
hospital.head()

,provider_id,provider_name,provider_city,provider_state,provider_zip_code,apc,apc_desc,capc_services,average_medicare_allowed_amount
0,10001,Southeast Alabama Medical Center,Dothan,AL,36301,5072,Level 2 Excision/ Biopsy/ Incision and Drainage,259,1038.45
1,10001,Southeast Alabama Medical Center,Dothan,AL,36301,5073,Level 3 Excision/ Biopsy/ Incision and Drainage,53,1792.60
2,10001,Southeast Alabama Medical Center,Dothan,AL,36301,5091,Level 1 Breast/Lymphatic Surgery and Related P...,27,2113.58
3,10001,Southeast Alabama Medical Center,Dothan,AL,36301,5092,Level 2 Breast/Lymphatic Surgery and Related P...,23,3737.14
4,10001,Southeast Alabama Medical Center,Dothan,AL,36301,5112,Level 2 Musculoskeletal Procedures,17,1029.46


In [26]:
# Assign every zipcode a CBSA
hospital_with_cbsa = pd.merge(hospital, cbsa, how='left', left_on='provider_zip_code', right_on='ZIP')

This is every state - hospital

In [27]:
hospital_with_cbsa

,provider_id,provider_name,provider_city,provider_state,provider_zip_code,apc,apc_desc,capc_services,average_medicare_allowed_amount,ZIP,CBSA
0,10001,Southeast Alabama Medical Center,Dothan,AL,36301,5072,Level 2 Excision/ Biopsy/ Incision and Drainage,259,1038.45,36301,20020
1,10001,Southeast Alabama Medical Center,Dothan,AL,36301,5073,Level 3 Excision/ Biopsy/ Incision and Drainage,53,1792.60,36301,20020
2,10001,Southeast Alabama Medical Center,Dothan,AL,36301,5091,Level 1 Breast/Lymphatic Surgery and Related P...,27,2113.58,36301,20020
3,10001,Southeast Alabama Medical Center,Dothan,AL,36301,5092,Level 2 Breast/Lymphatic Surgery and Related P...,23,3737.14,36301,20020
4,10001,Southeast Alabama Medical Center,Dothan,AL,36301,5112,Level 2 Musculoskeletal Procedures,17,1029.46,36301,20020
...,...,...,...,...,...,...,...,...,...,...,...
71065,670122,Houston Methodist The Woodlands Hospital,The Woodlands,TX,77385,5361,Level 1 Laparoscopy and Related Services,18,3866.82,77385,26420
71066,670122,Houston Methodist The Woodlands Hospital,The Woodlands,TX,77385,5373,Level 3 Urology and Related Services,11,1469.87,77385,26420
71067,670122,Houston Methodist The Woodlands Hospital,The Woodlands,TX,77385,5375,Level 5 Urology and Related Services,21,3356.89,77385,26420
71068,670122,Houston Methodist The Woodlands Hospital,The Woodlands,TX,77385,5431,Level 1 Nerve Procedures,12,1506.38,77385,26420


In [28]:
hospital_with_cbsa = hospital_with_cbsa.drop('ZIP', axis =1)

In [29]:
# Filter to just Nashville Hospitals
# This is ALL Nashville providers and procedures
nash_hospital = hospital_with_cbsa[hospital_with_cbsa.provider_zip_code.isin(zip_34980)]

In [30]:
# Filter down to the right apc
nash_hospital

,provider_id,provider_name,provider_city,provider_state,provider_zip_code,apc,apc_desc,capc_services,average_medicare_allowed_amount,CBSA
57520,440073,Maury Regional Hospital,Columbia,TN,38401,5491,Level 1 Intraocular Procedures,350,1660.34,30280
57521,440073,Maury Regional Hospital,Columbia,TN,38401,5491,Level 1 Intraocular Procedures,350,1660.34,34980
57574,440003,Sumner Regional Medical Center,Gallatin,TN,37066,5072,Level 2 Excision/ Biopsy/ Incision and Drainage,79,1089.32,34980
57575,440003,Sumner Regional Medical Center,Gallatin,TN,37066,5073,Level 3 Excision/ Biopsy/ Incision and Drainage,11,2010.12,34980
57576,440003,Sumner Regional Medical Center,Gallatin,TN,37066,5112,Level 2 Musculoskeletal Procedures,19,1091.34,34980
...,...,...,...,...,...,...,...,...,...,...
59312,440227,Tristar Stonecrest Medical Center,Smyrna,TN,37167,5374,Level 4 Urology and Related Services,16,2378.34,34980
59313,440227,Tristar Stonecrest Medical Center,Smyrna,TN,37167,5375,Level 5 Urology and Related Services,44,3091.00,34980
59314,440227,Tristar Stonecrest Medical Center,Smyrna,TN,37167,5415,Level 5 Gynecologic Procedures,12,3341.62,34980
59315,440227,Tristar Stonecrest Medical Center,Smyrna,TN,37167,5463,Level 3 Neurostimulator and Related Procedures,17,16612.08,34980


In [31]:
nash_hosp_5302 = nash_hospital[nash_hospital['apc'] == '5302']

USE BELOW TABLE FOR NASHVILLE HOSPITALS

In [32]:
nash_hosp_5302

,provider_id,provider_name,provider_city,provider_state,provider_zip_code,apc,apc_desc,capc_services,average_medicare_allowed_amount,CBSA
57584,440003,Sumner Regional Medical Center,Gallatin,TN,37066,5302,Level 2 Upper GI Procedures,114,1223.45,34980
57602,440006,Tristar Skyline Medical Center,Nashville,TN,37207,5302,Level 2 Upper GI Procedures,15,1248.62,34980
57883,440029,Williamson Medical Center,Franklin,TN,37067,5302,Level 2 Upper GI Procedures,18,1248.62,34980
58008,440039,Vanderbilt University Medical Center,Nashville,TN,37232,5302,Level 2 Upper GI Procedures,623,1205.62,34980
58050,440046,Tristar Horizon Medical Center,Dickson,TN,37055,5302,Level 2 Upper GI Procedures,13,1095.57,99999
58051,440046,Tristar Horizon Medical Center,Dickson,TN,37055,5302,Level 2 Upper GI Procedures,13,1095.57,34980
58190,440053,Saint Thomas Rutherford Hospital,Murfreesboro,TN,37129,5302,Level 2 Upper GI Procedures,55,1248.62,34980
58306,440065,Northcrest Medical Center,Springfield,TN,37172,5302,Level 2 Upper GI Procedures,29,1248.62,34980
58399,440073,Maury Regional Hospital,Columbia,TN,38401,5302,Level 2 Upper GI Procedures,254,1229.64,30280
58400,440073,Maury Regional Hospital,Columbia,TN,38401,5302,Level 2 Upper GI Procedures,254,1229.64,34980


In [33]:
# Filter to just New York Hospitals
# This is ALL New York providers and procedures
ny_hospital = hospital_with_cbsa[hospital_with_cbsa.provider_zip_code.isin(zip_35620)]

In [34]:
# Narrow down to correct APC
ny_hosp_5302 = ny_hospital[ny_hospital['apc'] == '5302']

USE BELOW TABLE FOR NEW YORK HOSPITALS

In [35]:
ny_hosp_5302

,provider_id,provider_name,provider_city,provider_state,provider_zip_code,apc,apc_desc,capc_services,average_medicare_allowed_amount,CBSA
38521,310001,Hackensack University Medical Center,Hackensack,NJ,07601,5302,Level 2 Upper GI Procedures,197,1538.63,35620
38559,310002,Newark Beth Israel Medical Center,Newark,NJ,07112,5302,Level 2 Upper GI Procedures,20,1528.38,35620
38572,310003,Palisades Medical Center,North Bergen,NJ,07047,5302,Level 2 Upper GI Procedures,22,1544.85,35620
38594,310005,Hunterdon Medical Center,Flemington,NJ,08822,5302,Level 2 Upper GI Procedures,53,1398.08,35620
38611,310006,St Mary's General Hospital,Passaic,NJ,07055,5302,Level 2 Upper GI Procedures,39,1544.85,35620
...,...,...,...,...,...,...,...,...,...,...
43457,330340,University Hospital - Stony Brook Southampton,Southampton,NY,11968,5302,Level 2 Upper GI Procedures,17,1589.69,35620
43539,330393,University Hospital,Stony Brook,NY,11794,5302,Level 2 Upper GI Procedures,494,1571.75,35620
43602,330395,St John's Episcopal Hospital At South Shore,Far Rockaway,NY,11691,5302,Level 2 Upper GI Procedures,17,1589.69,35620
43617,330399,St Barnabas Hospital,Bronx,NY,10457,5302,Level 2 Upper GI Procedures,25,1589.81,35620


Option 1 Stretch 1
Extend to other HCPCS/APCs
    Best place to start is to find an APC that is well-populated in the Hospital file
    Next find a well-populated HCPCS from the APC in the Physician/Other Supplier File
    How many patients do providers in zip A see compared to zip B?


In [36]:
# Find well populated APC nationwide
# Use 5114    Level 4 musculoskeletal procedures aka procedures for when you get older. Makes sense
hospital_with_cbsa['apc'].value_counts()

8011    3568
5072    3155
5113    3076
5361    3006
5114    2959
5341    2705
5302    2674
5375    2579
5112    2520
5373    2423
5374    2338
5431    2287
5191    2135
5073    2135
5491    2054
5193    1974
5091    1740
5223    1724
5092    1654
5414    1609
5192    1552
5115    1437
5154    1389
5362    1320
5153    1304
5194    1229
5222    1150
5232    1133
5415    1001
5165     952
5213     903
5313     842
5492     823
5303     800
5155     735
5164     653
5503     618
5464     604
5231     514
5462     489
5331     486
5463     446
5376     359
5627     278
5416     270
5377     258
5212     253
5224     253
5504     164
5471     149
5166     117
5116      80
5094      51
5432      50
5093      41
5200      40
5211       9
5493       3
Name: apc, dtype: int64

In [45]:
# crosswalk[ro]

In [38]:
hospital_5114 = hospital_with_cbsa[hospital_with_cbsa['apc'] == '5114']

USE TABLE BELOW FOR NATIONWIDE HOSPITALS UNDER APC 5114

In [39]:
# nationwide 5114 hospitals
hospital_5114

,provider_id,provider_name,provider_city,provider_state,provider_zip_code,apc,apc_desc,capc_services,average_medicare_allowed_amount,CBSA
6,10001,Southeast Alabama Medical Center,Dothan,AL,36301,5114,Level 4 Musculoskeletal Procedures,175,4399.54,20020
66,10005,Marshall Medical Center South,Boaz,AL,35957,5114,Level 4 Musculoskeletal Procedures,126,4433.83,22840
67,10005,Marshall Medical Center South,Boaz,AL,35957,5114,Level 4 Musculoskeletal Procedures,126,4433.83,13820
68,10005,Marshall Medical Center South,Boaz,AL,35957,5114,Level 4 Musculoskeletal Procedures,126,4433.83,23460
69,10005,Marshall Medical Center South,Boaz,AL,35957,5114,Level 4 Musculoskeletal Procedures,126,4433.83,10700
...,...,...,...,...,...,...,...,...,...,...
71031,670112,Cumberland Surgical Hospital,San Antonio,TX,78249,5114,Level 4 Musculoskeletal Procedures,53,4711.67,41700
71040,670116,Wise Health System,Fort Worth,TX,76177,5114,Level 4 Musculoskeletal Procedures,103,4878.58,19100
71051,670119,Providence Hospital Of North Houston Llc,Houston,TX,77090,5114,Level 4 Musculoskeletal Procedures,23,5031.05,26420
71055,670120,The Hospitals Of Providence Transmountain Campus,El Paso,TX,79911,5114,Level 4 Musculoskeletal Procedures,13,4497.41,21340


In [ ]:
# Get Nashville hospitals for code 5114


In [46]:
# Get all physicians under 5114 apc, code 63047 nationwide

In [40]:
# Get associated hcpcs codes from apc 
crosswalk_5114 = crosswalk[crosswalk['apc'] == 5114]
crosswalk_5114.drop(['national_unadjusted_copayment', 'si', 'relative_weight', 'minimum_unadjusted_copayment', 'note:_actual_copayments_would_be_lower_due_to_the_cap_on_copayments_at_the_inpatient_deductible_of_$1,408.00', '*_indicates_a_change', 'unnamed:_10'], axis = 1)

,hcpcs_code,short_descriptor,apc,payment_rate
399,11970,Replace tissue expander,5114.0,"$5,981.95"
710,20251,Open bone biopsy,5114.0,"$5,981.95"
741,20690,Apply bone fixation device,5114.0,"$5,981.95"
743,20693,Adjust bone fixation device,5114.0,"$5,981.95"
761,20900,Removal of bone for graft,5114.0,"$5,981.95"
...,...,...,...,...
9703,0275T,Perq lamot/lam lumbar,5114.0,"$5,981.95"
9722,0335T,Insj sinus tarsi implant,5114.0,"$5,981.95"
9868,0511T,Rmvl&rinsj sinus tarsi implt,5114.0,"$5,981.95"
12881,G0276,Pild/placebo control clin tr,5114.0,"$5,981.95"


In [41]:
# Nationwide providers under apc 5114
physician_5114_apc = physician_with_cbsa[physician_with_cbsa.hcpcs_code.isin(crosswalk_5114['hcpcs_code'])]

In [42]:
# Well populated hcpcs code from 5114 apc is 63047
physician_5114_apc['hcpcs_code'].value_counts()

63047    5399
29827    3963
63030    1058
29828     714
23430     684
         ... 
25825       1
28585       1
27696       1
27065       1
26842       1
Name: hcpcs_code, Length: 127, dtype: int64

Top 5 providers are
Orthopedic Surgery                      6811
Physician Assistant                     3383
Neurosurgery                            3000
Ambulatory Surgical Center              1557
Nurse Practitioner                       500

In [43]:
# Create dataframe containing only 63047 hcpcs codes
# Nationwide
physician_63047 = physician_5114_apc[(physician_5114_apc['hcpcs_code'] == '63047')]

USE BELOW TABLE FOR NATIONWIDE PHYSICIANS UNDER CODE 63047 
<b>
This will be comparing doctors themselves and thus not the overall cost of a procedure, just the doctors portion (most likely)

In [44]:
physician_63047

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_credentials,nppes_entity_code,nppes_provider_zip,nppes_provider_state,provider_type,place_of_service,hcpcs_code,bene_day_srvc_cnt,average_medicare_allowed_amt,CBSA
3877,1003018748,THOMAS,BRAD,MD,I,72205,AR,Neurosurgery,F,63047,67,751.756269,30780
5533,1003027392,CURT,BRADFORD,MD,I,45069,OH,Neurosurgery,F,63047,42,889.988095,17140
9637,1003050071,ROHDE,BREANNE,PA-C,I,98664,WA,Physician Assistant,F,63047,25,144.441200,38900
9747,1003050493,AUSCHWITZ,TYLER,NaN,I,74104,OK,Neurosurgery,F,63047,21,738.330000,46140
10394,1003055666,MILLER,JUSTIN,PA,I,49525,MI,Physician Assistant,F,63047,13,110.696923,24340
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10977918,1992961361,YOCUM,TONIA,PA-C,I,23601,VA,Physician Assistant,F,63047,81,80.035926,47260
10979900,1992968606,FARHADI,HOOMAN,MD,I,43210,OH,Neurosurgery,F,63047,11,545.405455,18140
10981823,1992983662,RENTFROW,COREY,P.A.-C,I,63122,MO,Physician Assistant,F,63047,66,106.054545,41180
10982091,1992986434,MARSH,ROBERT,"M.D., PH.D.",I,25701,WV,Neurosurgery,F,63047,15,1057.933333,26580


Option 1 Stretch 2
Compare Doctor Utilization for a procedure
    Filters for Physician/Other Supplier
    Entity Type = I
    Place of Service = O
    HCPCS = 99213 (or any office visit – 99211-99215 / 99201-99205)
    How many different patients does each specialty typically see in a year? How does that vary by CBSA?


In [ ]:
# Narrow down physicians to just office visits 
# HCPCS = 99213 (or any office visit – 99211-99215 / 99201-99205)

In [47]:
office_visit = ['99213', '99211', '99212','99213','99214','99215', '99201','99202','99203','99204','99205']

In [53]:
phys_office = physician_with_cbsa[(physician_with_cbsa['nppes_entity_code'] == 'I') & (physician_with_cbsa['place_of_service'] == 'O')]

In [54]:
phys_office

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_credentials,nppes_entity_code,nppes_provider_zip,nppes_provider_state,provider_type,place_of_service,hcpcs_code,bene_day_srvc_cnt,average_medicare_allowed_amt,CBSA
37,1003000142,KHALIL,RASHID,M.D.,I,43623,OH,Anesthesiology,O,99204,73,160.330000,45780
38,1003000142,KHALIL,RASHID,M.D.,I,43623,OH,Anesthesiology,O,99205,36,202.250000,45780
39,1003000142,KHALIL,RASHID,M.D.,I,43623,OH,Anesthesiology,O,99213,129,70.930000,45780
41,1003000142,KHALIL,RASHID,M.D.,I,43623,OH,Anesthesiology,O,99214,330,104.530000,45780
42,1003000142,KHALIL,RASHID,M.D.,I,43623,OH,Anesthesiology,O,99215,49,141.060000,45780
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10983883,1992999825,DESCHENES,GEOFFREY,M.D.,I,98101,WA,Otolaryngology,O,31575,63,127.160000,42660
10983885,1992999825,DESCHENES,GEOFFREY,M.D.,I,98101,WA,Otolaryngology,O,69210,104,52.807212,42660
10983887,1992999825,DESCHENES,GEOFFREY,M.D.,I,98101,WA,Otolaryngology,O,99203,88,117.260000,42660
10983889,1992999825,DESCHENES,GEOFFREY,M.D.,I,98101,WA,Otolaryngology,O,99204,105,176.900000,42660


In [56]:
phys_office = phys_office[phys_office['hcpcs_code'].isin(office_visit)]

In [59]:
phys_office_group = phys_office.groupby(['provider_type', 'CBSA']).sum().reset_index()

USE BELOW TABLE FOR OFFICE VISITS BY PROVIDER TYPE BY CBSA

In [61]:
phys_office_group

,provider_type,CBSA,npi,bene_day_srvc_cnt,average_medicare_allowed_amt
0,Addiction Medicine,10420,1285669481,58,69.510000
1,Addiction Medicine,10580,5516757423,844,273.590000
2,Addiction Medicine,10740,2612178626,188,129.660000
3,Addiction Medicine,11220,2854233032,252,102.360000
4,Addiction Medicine,11460,5008771056,388,472.670000
...,...,...,...,...,...
26982,Vascular Surgery,49620,38687740813,2430,2402.933333
26983,Vascular Surgery,49660,50337375139,2697,3410.577176
26984,Vascular Surgery,49700,6516740912,713,511.460000
26985,Vascular Surgery,49780,9269457888,764,735.770000


List all dataframes for CSVs

In [65]:
# dt.to_csv('C:/Users/abc/Desktop/file_name.csv')
# "C:\Users\toshi\Documents\NSS Documents\Projects\Python\healthcare-bluebook-red-team\Option_1_CSV"
nash_prov_43249.to_csv(r"C:\Users\toshi\Documents\NSS Documents\Projects\Python\healthcare-bluebook-red-team\Option_1_CSV\nash_prov_43249.csv", encoding='utf-8', index=False)
ny_prov_43249.to_csv(r"C:\Users\toshi\Documents\NSS Documents\Projects\Python\healthcare-bluebook-red-team\Option_1_CSV\ny_prov_43249.csv", encoding='utf-8', index=False)
nash_hosp_5302.to_csv(r"C:\Users\toshi\Documents\NSS Documents\Projects\Python\healthcare-bluebook-red-team\Option_1_CSV\nash_hosp_5302.csv", encoding='utf-8', index=False)
ny_hosp_5302.to_csv(r"C:\Users\toshi\Documents\NSS Documents\Projects\Python\healthcare-bluebook-red-team\Option_1_CSV\ny_hosp_5302.csv", encoding='utf-8', index=False)
hospital_5114.to_csv(r"C:\Users\toshi\Documents\NSS Documents\Projects\Python\healthcare-bluebook-red-team\Option_1_CSV\hospital_5114.csv", encoding='utf-8', index=False)
physician_63047.to_csv(r"C:\Users\toshi\Documents\NSS Documents\Projects\Python\healthcare-bluebook-red-team\Option_1_CSV\physician_63047.csv", encoding='utf-8', index=False)
phys_office_group.to_csv(r"C:\Users\toshi\Documents\NSS Documents\Projects\Python\healthcare-bluebook-red-team\Option_1_CSV\phys_office_group.csv", encoding='utf-8', index=False)